In [ ]:
import tensorflow as tf
import scipy.io as sio
sess = tf.InteractiveSession()
import os
import numpy as np

In [ ]:
# Define functions

def conv2d(x, W, dy ,dx):
  return tf.nn.conv2d(x, W, strides=[1, dx, dy, 1], padding='SAME')

def max_pool_axb(x,a,b):
  return tf.nn.max_pool(x, ksize=[1, a, b, 1],
                        strides=[1, a, b, 1], padding='SAME')

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [ ]:
# # Define the parameters of the network


# nx_conv1=4      # filter_width
# my_conv1=32     # filter_height
# dnx_conv1=1     # stride_width
# dmy_conv1=8     # stride_height

# nx_pool1=2      # pooling width
# my_pool1=2      # pooling height

# channels_conv1=12    # channels num

# nx_conv2=16
# my_conv2=2
# dnx_conv2=8
# dmy_conv2=1

# nx_pool2=2
# my_pool2=2


# channels_conv2=2

# size_input_fc1=int(128*128/dnx_conv1/dmy_conv1/dnx_conv2/dmy_conv2)*channels_conv2
# num_neurons_fc1=1024


In [ ]:
# Define the parameters of the network

nx_conv1=16      # filter_width (frequency)
my_conv1=4   # filter_height (time)
dnx_conv1=4    # stride_width
dmy_conv1=1    # stride_height

nx_pool1=4      # pooling width
my_pool1=1      # pooling height

channels_conv1=32    # channels num

nx_conv2=2
my_conv2=16
dnx_conv2=1
dmy_conv2=8

nx_pool2=2
my_pool2=1


channels_conv2=16

size_input_fc1=int(128*128*channels_conv2/dnx_conv1/dmy_conv1/dnx_conv2/dmy_conv2/nx_pool1/my_pool1/nx_pool2/my_pool2)
num_neurons_fc1=64

print(size_input_fc1)

In [ ]:
# x = tf.placeholder("float", shape=[None, 128*128])
# y_ = tf.placeholder("float", shape=[None, 4])


# x_image = tf.reshape(x, [-1,128,128,1])

# W_conv1 = weight_variable([nx_conv1,my_conv1,1,channels_conv1])
# b_conv1 = bias_variable([channels_conv1])


# h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, dnx_conv1, dmy_conv1 ) + b_conv1)
# h_pool1 = max_pool_axb(h_conv1, nx_pool1, my_pool1)


# W_conv2 = weight_variable([nx_conv2,my_conv2, channels_conv1, channels_conv2])
# b_conv2 = bias_variable([channels_conv2])

# h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, dnx_conv2, dmy_conv2 ) + b_conv2)
# h_pool2 = max_pool_axb(h_conv2, nx_pool2, my_pool2)



# W_fc1 = weight_variable([size_input_fc1, num_neurons_fc1])
# b_fc1 = bias_variable([num_neurons_fc1])

# h_pool2_flat = tf.reshape(h_pool2, [-1, size_input_fc1])
# h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


# W_fc2 = weight_variable([num_neurons_fc1, 4])
# b_fc2 = bias_variable([4])



# keep_prob = tf.placeholder("float")
# h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)



# cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
# train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))


In [ ]:
x = tf.placeholder("float", shape=[None, 128*128])
y_ = tf.placeholder("float", shape=[None, 2])


x_image = tf.reshape(x, [-1,128,128,1])

W_conv1 = weight_variable([nx_conv1,my_conv1,1,channels_conv1])
b_conv1 = bias_variable([channels_conv1])


h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, dnx_conv1, dmy_conv1 ) )
h_pool1 = max_pool_axb(h_conv1, nx_pool1, my_pool1)


W_conv2 = weight_variable([nx_conv2,my_conv2, channels_conv1, channels_conv2])
b_conv2 = bias_variable([channels_conv2])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, dnx_conv2, dmy_conv2 ) )
h_pool2 = max_pool_axb(h_conv2, nx_pool2, my_pool2)


W_fc1 = weight_variable([size_input_fc1, num_neurons_fc1])
b_fc1 = bias_variable([num_neurons_fc1])

h_pool2_flat = tf.reshape(h_pool2, [-1, size_input_fc1])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) )


W_fc2 = weight_variable([num_neurons_fc1, 2])
b_fc2 = bias_variable([2])


keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2))



cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))


In [ ]:
#sess.run(tf.initialize_all_variables())  


In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
saver = tf.train.Saver()


In [ ]:
step_size=200

# Read osu! data

dataList=os.listdir('C:/OSU/SongMat/Train/')

for n in range(int(np.size(dataList)/10)):
    inputOsu=np.empty([0,128*128]);
    outputOsu=np.empty([0,4]);
    for k in range(n*10,n*10+8):
        
        data=sio.loadmat('C:/OSU/SongMat/Train/'+dataList[k])
        inputOsu=np.append(inputOsu,data['input1'].reshape(-1,128*128),axis=0)
        outputOsu=np.append(outputOsu,data['target1'].reshape(-1,4),axis=0)
               
    outputOsu2=np.zeros([outputOsu.shape[0],2]);
    outputOsu2[:,0]=outputOsu[:,0]+outputOsu[:,1]+outputOsu[:,2]
    outputOsu2[:,1]=outputOsu[:,3]
        
    N=inputOsu.shape[0]
    print(N)
    
    #shuffle the order
    order = np.arange(N-step_size)
    np.random.shuffle(order)
    
    print("test accuracy %g"%accuracy.eval(feed_dict={x: inputOsu[-300:,:], y_: outputOsu2[-300:,:], keep_prob: 1.0}))
    print(y_conv.eval(feed_dict={x: inputOsu[200:210:1,:], keep_prob: 1.0}))
    print(W_fc1.eval())
    
    for i in range(N-step_size):
        if i%step_size == 0:
            train_accuracy = accuracy.eval(feed_dict={x: inputOsu[order[i:i+step_size],:], y_: outputOsu2[order[i:i+step_size],:], keep_prob: 1.0})
            print("step %d, training accuracy %g" %(i, train_accuracy))
        train_step.run(feed_dict={x: inputOsu[order[i:i+step_size],:], y_: outputOsu2[order[i:i+step_size],:], keep_prob: 0.7})
    
    
   


In [ ]:
save_path = saver.save(sess, "D:/OSU/results/save/",global_step=n)

In [ ]:
print ("test accuracy %g"%accuracy.eval(feed_dict={
  x: inputOsu[:500,:], y_: outputOsu[:500,:], keep_prob: 1.0}))

In [ ]:
# Save the output of new maps 

dataList=os.listdir('D:/OSU/SongMat/Create/')
# inputOsuNew=np.empty([0,128*128]);
for i in range(np.size(dataList)):
    if dataList[i][0]!='y':
        data=sio.loadmat('D:/OSU/SongMat/Create/'+dataList[i])
        inputOsuNew=data['input'].reshape(-1,128*128)
        if inputOsuNew.shape[0]>2000:
            y_result=y_conv.eval(feed_dict={x: inputOsuNew[:2000,:], keep_prob: 1.0})
        else:
            y_result=y_conv.eval(feed_dict={x: inputOsuNew[:,:], keep_prob: 1.0})
        data_save={'y':y_result}
        sio.savemat('D:/OSU/SongMat/Create/y_'+dataList[i],data_save)



In [ ]:
print(y_conv.eval(feed_dict={x: inputOsu[:200,:], keep_prob: 1.0}))

In [ ]:
data_save={'y':y_result}
sio.savemat('D:/OSU/results/innocence/y_innocence.mat',data_save)

In [ ]:
checkpoint_dir='D:/OSU/results/innocence/save/'
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)  
saver.restore(sess, ckpt.model_checkpoint_path)  

In [ ]:
saver2 = tf.train.Saver()

In [ ]:
checkpoint_dir='/Users/dongqihan/Desktop/FrOstNova/save/'
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)  
saver2.restore(sess, ckpt.model_checkpoint_path)  

In [ ]:
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess_tmp = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess_tmp.run(c))

In [ ]:
data_test=sio.loadmat('/Users/dongqihan/Desktop/FrOstNova/granat.mat')
inputOsu_test=data_test['input'].reshape(-1,128*128)
y_result_test=y_conv.eval(feed_dict={x: inputOsu_test, keep_prob: 1.0})
data={'y':y_result_test}
sio.savemat('/Users/dongqihan/Desktop/FrOstNova/y_memory2granat.mat',data)